<a href="https://colab.research.google.com/github/dhruv369/AI/blob/master/MNIST_Digit_addition_CNN_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Assignment - PyTorch 

Write a neural network that can:

take 2 inputs:
an image from the MNIST dataset (say 5), and
a random number between 0 and 9, (say 7)
and gives two outputs:
the "number" that was represented by the MNIST image (predict 5), and
the "sum" of this number with the random number and the input image to the network (predict 5 + 7 = 12)

assign-1.png     

you can mix fully connected layers and convolution layers
you can use one-hot encoding to represent the random number input as well as the "summed" output.

Random number (7) can be represented as 0 0 0 0 0 0 0 1 0 0
Sum (13) can be represented as:
0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
0b1101 (remember that 4 digits in binary can at max represent 15, so we may need to go for 5 digits. i.e. 10010


In [1]:
# Pytorch's tensors are similar to Numpy's ndarrays
!pip install torch

#Import libs

In [ ]:
import torch
import torchvision #datasets, models, transforms utils
import torchvision.transforms as transforms
from torch.utils.data import Dataset


#Prepare data set for first 0 - 9 input

In [ ]:
import numpy as np 
aa = [[1,1],[2,2],[3,3]]
train_data = np.array([[1.0,1.0]])
train_targets = np.array([2.0])
train_data_digit = np.array([])
for i in range(3,10):
    print(i)
    train_data_digit = np.append(train_data,[[i,i]],axis=0)
    train_digit_targets= np.append([0],[i+i])

3
4
5
6
7
8
9


In [ ]:
for i in train_data_digit:
  print(i)

[1. 1.]
[9. 9.]


In [ ]:
class MyDataset(Dataset):
  def __init__(self):
    self.data = (1,2,3,4,5,6,7,8,9,0)
  
  def __getitem__(self,index):
    return self.data[index]

  def __len__(self):
    return len(self.data)

myData = MyDataset()

for ii in myData:
  print(ii)

1
2
3
4
5
6
7
8
9
0


In [ ]:
# train_set = torchvision.datasets.FashionMNIST(root='./data',train=True,download=True,transform=transforms.Compose([transforms.ToTensor()]))

#Make NN

In [ ]:
import torch.nn.functional as F

class Network(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5) 
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
    self.fc1 = nn.Linear(in_features=(12 * 4 * 4)+1, out_features=120)
    self.fc2 = nn.Linear(in_features=120, out_features=60)
    self.out_1 = nn.Linear(in_features=60, out_features=10)
    self.fc3 = nn.Liner(in_features=2,out_features=20)
    self.fc4 = nn.Liner(in_features=2,out_features=20)
    self.out_2 = nn.Linear(in_features=20, out_features=1)

  
  def forward(self, input_image, input_digit):
    # input layer
    x = input_image
    d = input_digit

    # conv1 layer
    x = self.conv1(x)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2, stride=2) # 28 | 24 | 12

    # conv2 layer
    x = self.conv2(x)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2, stride=2) # 12 | 8 | 4 >> 12x4x4

    # reshapre
    x = x.reshape(-1, (12 * 4 * 4))
    # print(x.shape,d.shape) 
    # x = torch.cat((x, d.reshape(-1,1)), dim=1)

    # fc1 layer
    x = self.fc1(x)
    x = F.relu(x)

    # fc2 layer
    x = self.fc2(x)
    x = F.relu(x)

    # output layer
    x = self.out_1(x)
    x = F.softmax(x, dim=1)

    # fc3 layer
    x = self.fc3(x)
    x = F.relu(x)

    #fc4 layer
    x = self.fc4(x)
    x = F.relu(x)

    # output layer
    x = self.out_2(x)
    x = F.softmax(x, dim=1)
    return x

In [ ]:
network = Network()
for name, param in network.named_parameters():
  print(name, '\t\t', param.shape)

train_loader = torch.utils.data.DataLoader(train_set,batch_size=2)
train_loader_digits = torch.utils.data.DataLoader(myData,batch_size=2)
optimizer = torch.optim.Adam(network.parameters(),lr=0.001)

for epoch in range(10):
  digit_iterator = iter(train_loader_digits)
  total_loss = 0
  total_correct = 0

  for i, image_data1 in enumerate(train_loader):
    try:
      data2 = next(digit_iterator)
    except StopIteration:
      digit_iterator = iter(train_loader_digits)
      data2 = next(digit_iterator)

    images, labels = image_data1
    digits = data2
    preds = network(images,data2)
    loss = F.cross_entropy(preds,labels)
    loss_digit = F.mse_loss(preds,labels)

    optimizer.zero_grad()
    loss.backward()
    loss_digiti.backward()
    optimizer.step()

    total_loss =  loss.item()
    total_correct += get_num_correct(preds,labels)

  print(
      "epoch", epoch, 
      "total_correct:", total_correct, 
      "loss:", total_loss
  )

conv1.weight 		 torch.Size([6, 1, 5, 5])
conv1.bias 		 torch.Size([6])
conv2.weight 		 torch.Size([12, 6, 5, 5])
conv2.bias 		 torch.Size([12])
fc1.weight 		 torch.Size([120, 193])
fc1.bias 		 torch.Size([120])
fc2.weight 		 torch.Size([60, 120])
fc2.bias 		 torch.Size([60])
out.weight 		 torch.Size([10, 60])
out.bias 		 torch.Size([10])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


RuntimeError: ignored